In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import statsmodels.api as sm

pd.set_option('float_format', '{:6.2f}'.format)
sns.set(style='ticks')

import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("wage2.txt",na_values='.')

In [3]:
df.head()

,wage,hours,iq,kww,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage
0,769,40,93,35,12,11,2,31,1,0,0,1,1,2.00,8.00,8.00,6.65
1,808,50,119,41,18,11,16,37,1,0,0,1,1,nan,14.00,14.00,6.69
2,825,40,108,46,14,11,9,33,1,0,0,1,1,2.00,14.00,14.00,6.72
3,650,40,96,32,12,13,7,32,1,0,0,1,4,3.00,12.00,12.00,6.48
4,562,40,74,27,11,14,5,34,1,0,0,1,10,6.00,6.00,11.00,6.33


In [4]:
#2
df[['educ','exper','tenure','black']].describe()

,educ,exper,tenure,black
count,935.00,935.00,935.00,935.00
mean,13.47,11.56,7.23,0.13
std,2.20,4.37,5.08,0.33
min,9.00,1.00,0.00,0.00
25%,12.00,8.00,3.00,0.00
50%,12.00,11.00,7.00,0.00
75%,16.00,15.00,11.00,0.00
max,18.00,23.00,22.00,1.00


In [5]:
#3
from statsmodels.regression.linear_model import OLS
model = OLS(df.lwage , sm.add_constant(df[['educ','exper','tenure','black']]))
result = model.fit(cov_type='HC0') 
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     52.93
Date:                Wed, 18 Dec 2019   Prob (F-statistic):           3.29e-40
Time:                        14:00:06   Log-Likelihood:                -424.18
No. Observations:                 935   AIC:                             858.4
Df Residuals:                     930   BIC:                             882.6
Df Model:                           4                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6125      0.115     48.987      0.0

#4
Все коэффициенты являются значимыми на 5% уровне значимости, так как у всех коэффицентов, z статистика имеет достаточно большое значение, у нас H0: b0=0 H1: b0!=0, и так для всех коэффициентов, принимаем гипотезу H0 при значение z статистика на интервале (-1.96,1.96)

#5
при увеличении образования на 1 год, зарплата увеличится на (e^b1-1)*100%= 7,14% в среднем при прочих равных 

#6
В данном уравнении существует эндогенность, из-за того что, наша зарплата может повлиять на наше образование, то есть если у тебя достаточно средств, таким образом ты можешь оплатить себе образование, чтобы в дальнейшем твоя зарплата повысилась, то есть существует зависимость между зп и образование. Занижается, изначально было какое-то образование, далее после повышения зарплаты можно воздействовать на твое образование, то есть можно получить дополнительное образование, если имеешь достатчоное количество средств, при этом получается, что после повышения своей квалификации у тебя образовалось какое-то значение в образовании, выше чем изначальное, когда зп повлияла на образование, таким образом в данный момент твое образование выше, чем начальное, но получается что коэффициент при образовании должен уменьшиться, потому что образование повысилось, следовательно коэффициент занижен.

#7
переменная sibs является экзогенной, потому что наша зарплата которую мы получаем, не имеет особо связи с количество братьев и сестер, так как решения наших родителей какое количество детей иметь, скорее всего не влияет на нашу зарплату, и зарплата в большой степени зависит от твоих потребностей и желаний, какой количество усилий и времени ты готов это посвятить.

In [6]:
#8
df[['sibs','educ']].corr()

,sibs,educ
sibs,1.00,-0.24
educ,-0.24,1.00


из показателя видно, что корреялция существует между sibs и educ

In [7]:
df[['sibs','educ']].cov()

,sibs,educ
sibs,5.32,-1.21
educ,-1.21,4.83


Смещение коэф инструмента происходит, когда у нас ковариация между инструментом и регрессором, достаточная маленькая, при таком условии инструмент слабый, в данном случае ковариация между переменными не значительно низкая, поэтому можно считать инструмент относительно сильным, использовать можно

In [8]:
#9
from linearmodels.iv import IV2SLS
formula = 'lwage ~ 1 + exper + tenure + black + [educ ~ sibs]'
mod = IV2SLS.from_formula(formula, df)
reslutIV = mod.fit(cov_type = 'robust')
print(reslutIV.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1685
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1650
No. Observations:                 935   F-statistic:                    101.47
Date:                Wed, Dec 18 2019   P-value (F-stat)                0.0000
Time:                        14:00:10   Distribution:                  chi2(4)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      5.2160     0.5174     10.081     0.0000      4.2019      6.2301
exper          0.0209     0.0083     2.5215     0.01

#10
мы видим, что переменная sibs повлияла на коэффициент, перед educ, изначально был коэффициент занижен, он равнялся 0.07, теперь 0.09, это говорит о том, что существует эндогенность между зп и educ, то есть зп могла повлиять на образования

In [9]:
#11
model = OLS(df.educ , sm.add_constant(df.sibs))
result2 = model.fit(cov_type='HC0') 
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     52.93
Date:                Wed, 18 Dec 2019   Prob (F-statistic):           3.29e-40
Time:                        14:00:12   Log-Likelihood:                -424.18
No. Observations:                 935   AIC:                             858.4
Df Residuals:                     930   BIC:                             882.6
Df Model:                           4                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6125      0.115     48.987      0.0

In [12]:
df['predictions_educ'] = 14.1388 + -0.2279*df['sibs']

In [13]:
model = OLS(df.lwage , sm.add_constant(df[['predictions_educ','exper','tenure','black']]))
result = model.fit(cov_type='HC0')
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     24.38
Date:                Wed, 18 Dec 2019   Prob (F-statistic):           3.30e-19
Time:                        14:00:32   Log-Likelihood:                -474.00
No. Observations:                 935   AIC:                             958.0
Df Residuals:                     930   BIC:                             982.2
Df Model:                           4                                         
Covariance Type:                  HC0                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                5.7710      0.340  

#12
изначально ковариация экзогенных переменных с ошибкой = 0, но при этом ковариация с эндогенной переменной с ошибкой не 0, и cov(sibs,u)= 0, оценили модель, но при этом в educ теперь есть какие-то ошибки, когда делаем второй шаг, переменная educ, теперь экзогенная, но из-за того, что мы оценивали с ошибками, ковариация этой ошибки с изначальными экзогенными переменнами может быть не 0 => они могут стать эндогенными

#13
в 9 пункте при увеличении образования на 1 год, зарпалата увеличивается на (exp(0.0936)-1)*100%=9,8% при прочих равных. в 11 пункте при увеличении образования на 1 год, зарплата увеличивается на (exp(0.07)-1)*100%=7,25%, таким образом во втором случае у нас коэффициент при образовании занижен, это значит, что в 1 случае зарплата увеличивается больше при увеличении на 1 год в образовании. При этом у нас R^2 adj. в правильном случае имеет большее значение по сравнению с неправильной моделью.

#14 
пропущенной переменной может выступать проживание в большом городе "urban", с большой долей вероятности в большом городе средняя зарплата будет выше, чем в маленьком городе.

In [109]:
formula = 'lwage ~ 1 + exper + tenure + black + urban + [educ ~ sibs]'
mod = IV2SLS.from_formula(formula, df)
reslutIV = mod.fit(cov_type = 'robust')
print(reslutIV.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.2176
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2134
No. Observations:                 935   F-statistic:                    174.96
Date:                Sat, Dec 14 2019   P-value (F-stat)                0.0000
Time:                        20:38:24   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      5.2660     0.4961     10.614     0.0000      4.2935      6.2384
exper          0.0190     0.0080     2.3703     0.01

Неправильная модель

In [107]:
model = OLS(df.lwage , sm.add_constant(df[['predictions_educ','exper','tenure','black','urban']]))
result = model.fit(cov_type='HC0')
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.138
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     33.17
Date:                Sat, 14 Dec 2019   Prob (F-statistic):           3.34e-31
Time:                        20:37:47   Log-Likelihood:                -447.99
No. Observations:                 935   AIC:                             908.0
Df Residuals:                     929   BIC:                             937.0
Df Model:                           5                                         
Covariance Type:                  HC0                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                5.7466      0.329  

Видим что коэффициенты при переменных в двух моделях достаточно сильно отличаются, например urban (если мы живем в большом городе ) увеличивает зарплату в правильной модели (exp(0.1870)-1)*100=20,5% и educ увеличение на 1 год образования увеличивает зарпалату на(exp(0.081)-1)*100=8,5%. в неправильной модели 22,1% и 6,1% соотвественно.
То есть различие в показателях имеются.